安裝必要套件

In [ ]:
!pip install tensorflow scikit-learn pandas matplotlib tqdm

掛載 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定雲端硬碟目錄
import os
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'
os.makedirs(MODEL_DIR, exist_ok=True)

Mounted at /content/drive


In [ ]:
!pip install optuna optuna-integration[tfkeras] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.4 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow scikit-learn matplotlib python-docx -q

In [ ]:
!pip install optuna

In [ ]:
!pip install tensorflow optuna

In [ ]:
!pip install tensorflow scikit-learn pandas matplotlib tqdm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

下載並載入 C-MAPSS 資料集（FD001)

In [ ]:
# 上傳 kaggle.json（只需做一次）
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"billlin01","key":"c42c112425b5d48a2926ec740846bfc2"}'}

設定 Kaggle API 權限並下載資料

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# 以 NASA C-MAPSS 資料為例
# 替換成你要的 Kaggle dataset slug（網址最後一段）
!kaggle datasets download -d behrad3d/nasa-cmaps
!unzip -o nasa-cmaps.zip -d cmapss_data

Dataset URL: https://www.kaggle.com/datasets/behrad3d/nasa-cmaps
License(s): CC0-1.0
  0% 0.00/12.3M [00:00<?, ?B/s]
100% 12.3M/12.3M [00:00<00:00, 856MB/s]
Archive:  nasa-cmaps.zip
  inflating: cmapss_data/CMaps/Damage Propagation Modeling.pdf  
  inflating: cmapss_data/CMaps/RUL_FD001.txt  
  inflating: cmapss_data/CMaps/RUL_FD002.txt  
  inflating: cmapss_data/CMaps/RUL_FD003.txt  
  inflating: cmapss_data/CMaps/RUL_FD004.txt  
  inflating: cmapss_data/CMaps/readme.txt  
  inflating: cmapss_data/CMaps/test_FD001.txt  
  inflating: cmapss_data/CMaps/test_FD002.txt  
  inflating: cmapss_data/CMaps/test_FD003.txt  
  inflating: cmapss_data/CMaps/test_FD004.txt  
  inflating: cmapss_data/CMaps/train_FD001.txt  
  inflating: cmapss_data/CMaps/train_FD002.txt  
  inflating: cmapss_data/CMaps/train_FD003.txt  
  inflating: cmapss_data/CMaps/train_FD004.txt  
  inflating: cmapss_data/CMaps/x.txt  


讀取資料夾路徑

In [ ]:
import pandas as pd

train = pd.read_csv('/content/cmapss_data/CMaps/train_FD001.txt', sep='\s+', header=None)
print(train.head())

   0   1       2       3      4       5       6        7        8      9   \
0   1   1 -0.0007 -0.0004  100.0  518.67  641.82  1589.70  1400.60  14.62   
1   1   2  0.0019 -0.0003  100.0  518.67  642.15  1591.82  1403.14  14.62   
2   1   3 -0.0043  0.0003  100.0  518.67  642.35  1587.99  1404.20  14.62   
3   1   4  0.0007  0.0000  100.0  518.67  642.35  1582.79  1401.87  14.62   
4   1   5 -0.0019 -0.0002  100.0  518.67  642.37  1582.85  1406.22  14.62   

   ...      16       17       18      19    20   21    22     23     24  \
0  ...  521.66  2388.02  8138.62  8.4195  0.03  392  2388  100.0  39.06   
1  ...  522.28  2388.07  8131.49  8.4318  0.03  392  2388  100.0  39.00   
2  ...  522.42  2388.03  8133.23  8.4178  0.03  390  2388  100.0  38.95   
3  ...  522.86  2388.08  8133.83  8.3682  0.03  392  2388  100.0  38.88   
4  ...  522.19  2388.04  8133.80  8.4294  0.03  393  2388  100.0  38.90   

        25  
0  23.4190  
1  23.4236  
2  23.3442  
3  23.3739  
4  23.4044  

[5 rows

資料預處理（選感測器、平滑、標準化並存scaler）

In [ ]:
import random
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import shap

GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'
os.makedirs(MODEL_DIR, exist_ok=True)
max_rul = 130

columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]

def load_and_label(filepath, max_rul=130):
    df = pd.read_csv(filepath, sep='\s+', header=None)
    df.columns = columns
    df['RUL_linear'] = df.groupby('unit')['time'].transform('max') - df['time']
    df['RUL_linear'] = df['RUL_linear'].clip(upper=max_rul)
    return df

train = load_and_label('/content/cmapss_data/CMaps/train_FD001.txt', max_rul=max_rul)

# ---- 1. 挑掉變異太低的sensor ----
sensor_cols = [f's{i}' for i in range(1, 22)]
sensor_cols_keep = [col for col in sensor_cols if train[col].std() >= 1e-3]  # 防止有無變異sensor

# ---- 2. 分段線性退化 RUL標註 (你的 Algorithm1 不變) ----
def generate_piecewise_rul_algorithm1(df, feature_cols, window_size=12, threshold=0.2, max_rul=130, verbose=False):
    result_dfs = []
    for unit_id, group in df.groupby('unit'):
        group = group.sort_values('time').reset_index(drop=True)
        sensor_data = group[feature_cols].values
        num_cycles = len(group)
        num_windows = num_cycles // window_size

        if num_windows < 3:
            irul = num_cycles
            group['iRUL'] = irul
            group['RUL_piecewise'] = np.clip(irul - group['time'], 0, max_rul)
            result_dfs.append(group)
            continue

        centroids = [np.mean(sensor_data[i * window_size:(i + 1) * window_size], axis=0)
                     for i in range(num_windows)]
        base = centroids[0]  # 第一段健康期
        degradation_found = False

        for i in range(2, num_windows):  # 從 w3 開始比對
            dist_sq = np.sum((centroids[i] - base) ** 2)
            if verbose:
                print(f"[Unit {unit_id}] Compare w1 to w{i+1}: Dist^2 = {dist_sq:.4f}")
            if dist_sq >= threshold:
                degradation_start = i * window_size
                irul = num_cycles - degradation_start
                degradation_found = True
                break

        if not degradation_found:
            irul = num_cycles

        rul_piecewise = [
            irul if t <= (num_cycles - irul) else irul - (t - (num_cycles - irul))
            for t in range(num_cycles)
        ]
        group['iRUL'] = irul
        group['RUL_piecewise'] = np.clip(rul_piecewise, 0, max_rul)
        result_dfs.append(group)
    return pd.concat(result_dfs, ignore_index=True)

train = generate_piecewise_rul_algorithm1(
    train, sensor_cols_keep, window_size=12, threshold=0.2, max_rul=max_rul, verbose=False
)

# ---- 3. SHAP + 皮爾森選sensor (保證只選8個) ----
rf = RandomForestRegressor(n_estimators=100, random_state=GLOBAL_SEED)
rf.fit(train[sensor_cols_keep], train['RUL_piecewise'])

explainer = shap.Explainer(rf, train[sensor_cols_keep].iloc[:500])
shap_values = explainer(train[sensor_cols_keep].iloc[:500])
mean_shap = np.abs(shap_values.values).mean(axis=0)
shap_scores = pd.Series(mean_shap, index=sensor_cols_keep).sort_values(ascending=False)
top8_shap = shap_scores.head(8).index.tolist()

corrs = train[sensor_cols_keep + ['RUL_piecewise']].corr()['RUL_piecewise'].abs().sort_values(ascending=False)
top8_pearson = corrs.drop('RUL_piecewise').head(8).index.tolist()

selected_sensors = []
for s in top8_shap:
    if s in top8_pearson and s not in selected_sensors:
        selected_sensors.append(s)
for s in top8_pearson:
    if s not in selected_sensors:
        selected_sensors.append(s)
    if len(selected_sensors) == 8:
        break
print("最終選用的8個感測器:", selected_sensors)

features = selected_sensors + ['op1', 'op2', 'op3']

# ---- 4. 單一指數平滑 (alpha=0.2) ----
def exponential_smoothing(series, alpha=0.2):
    result = [series.iloc[0]]
    for n in range(1, len(series)):
        result.append(alpha * series.iloc[n] + (1 - alpha) * result[-1])
    return pd.Series(result, index=series.index)

for col in features:
    train[col] = train.groupby('unit')[col].transform(lambda x: exponential_smoothing(x, alpha=0.2))

final_features = features

# ---- 5. 標準化 ----
scaler = StandardScaler()
train[final_features] = scaler.fit_transform(train[final_features])

# ---- 6. 儲存 ----
joblib.dump(scaler, f'{MODEL_DIR}/scaler_preprocessed.joblib')
joblib.dump(final_features, f'{MODEL_DIR}/feature_names.pkl')
train.to_csv(f'{MODEL_DIR}/train_with_piecewise_rul.csv', index=False)

print('🚩 已完成分段線性RUL資料與所有特徵處理，檔案儲存：')
print(f'- {MODEL_DIR}/train_with_piecewise_rul.csv')


 99%|===================| 497/500 [01:02<00:00]       

最終選用的8個感測器: ['s11', 's4', 's12', 's7', 's21', 's15', 's20', 's17']
🚩 已完成分段線性RUL資料與所有特徵處理，檔案儲存：
- /content/drive/MyDrive/rul_diffusion_fd001/train_with_piecewise_rul.csv


產生 X_train 滑動視窗序列

In [ ]:
import numpy as np
import joblib
import pandas as pd

# 參數設定
SEQ_LEN = 32
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'

# 載入先前前處理後的特徵名稱
final_features = joblib.load(f'{MODEL_DIR}/feature_names.pkl')

# 載入資料（和前處理結果保持一致）
train = pd.read_csv(f'{MODEL_DIR}/train_with_piecewise_rul.csv')

# 建構滑動序列
X_list, y_list, unit_list = [], [], []
for unit in train['unit'].unique():
    df_unit = train[train['unit'] == unit]
    arr = df_unit[final_features].values
    rul_piecewise = df_unit['RUL_piecewise'].values  # label用piecewise
    # 產生滑動視窗序列
    for i in range(len(arr) - SEQ_LEN + 1):
        X_list.append(arr[i:i + SEQ_LEN])
        y_list.append(rul_piecewise[i + SEQ_LEN - 1])
        unit_list.append(unit)

X_all = np.stack(X_list).astype(np.float32)
y_all = np.array(y_list).astype(np.float32)
unit_all = np.array(unit_list).astype(int)

print("滑動視窗完成:")
print("X_all shape:", X_all.shape)  # (N, 32, 11)
print("y_all shape:", y_all.shape)  # (N,)

# 儲存資料（供後續encoder/joint train用）
np.save(f'{MODEL_DIR}/X_all.npy', X_all)
np.save(f'{MODEL_DIR}/y_all.npy', y_all)
np.save(f'{MODEL_DIR}/unit_all.npy', unit_all)  # optional, 分組/val用


滑動視窗完成:
X_all shape: (17531, 32, 11)
y_all shape: (17531,)


Stage 1 Encoder自監督預訓練，儲存模型到雲端

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.optimizers import Adam
import os, random

GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)

SEQ_LEN = 32
D = 16
T = 1000
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'
os.makedirs(MODEL_DIR, exist_ok=True)

N_SENSORS = 11  # 你的特徵數
BATCH_SIZE = 128
EPOCHS_STAGE1 = 300

# Stage1完整模型（輸出復原序列）
def build_encoder_full(input_dim, embed_dim, seq_len):
    input_x = Input(shape=(seq_len, input_dim), name='input_1')
    input_tau = Input(shape=(embed_dim,), name='input_2')
    t_proj = layers.Dense(input_dim)(input_tau)
    t_proj_exp = layers.Reshape((1, input_dim))(t_proj)
    x = layers.Add()([input_x, t_proj_exp])
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    output = layers.Conv1D(input_dim, 1, padding='same')(x)  # 預測噪音序列
    return Model([input_x, input_tau], output)

encoder_full = build_encoder_full(N_SENSORS, D, SEQ_LEN)
encoder_full.compile(optimizer=Adam(learning_rate=5e-4), loss='mse')

# Diffusion Beta schedule (cosine)
def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = np.linspace(0, timesteps, steps)
    f = np.cos(((x / timesteps) + s) / (1 + s) * np.pi / 2) ** 2
    alphas_cumprod = f / f[0]
    betas = np.clip(1 - (alphas_cumprod[1:] / alphas_cumprod[:-1]), 0, 0.999)
    return betas

betas = cosine_beta_schedule(T, s=0.008)
alphas = 1 - betas
alphas_cumprod = np.cumprod(alphas)

# 時間嵌入函數
def get_timestep_embedding(timesteps, dim=D):
    timesteps = tf.convert_to_tensor(timesteps, dtype=tf.float32)
    timesteps = tf.reshape(timesteps, [-1, 1])
    half_dim = dim // 2
    emb = tf.math.log(10000.0) / (half_dim - 1)
    emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
    emb = timesteps * emb
    emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
    return emb

# Diffusion資料生成器
def diffusion_dataset(X, batch_size, T, alphas_cumprod, D):
    def generator():
        dataset_size = len(X)
        while True:
            idxs = np.random.permutation(dataset_size)
            for i in range(0, dataset_size, batch_size):
                batch_idx = idxs[i:i+batch_size]
                x_start = X[batch_idx]
                b = len(x_start)
                t = np.random.randint(1, T + 1, size=b)
                tau = get_timestep_embedding(t, D).numpy()
                noise = np.random.randn(*x_start.shape).astype(np.float32)
                sqrt_alpha = np.sqrt(alphas_cumprod[t - 1])[:, None, None]
                sqrt_one_minus_alpha = np.sqrt(1 - alphas_cumprod[t - 1])[:, None, None]
                x_t = sqrt_alpha * x_start + sqrt_one_minus_alpha * noise
                yield {"input_1": x_t, "input_2": tau}, noise
    output_signature = (
        {"input_1": tf.TensorSpec(shape=(None, SEQ_LEN, N_SENSORS), dtype=tf.float32),
         "input_2": tf.TensorSpec(shape=(None, D), dtype=tf.float32)},
        tf.TensorSpec(shape=(None, SEQ_LEN, N_SENSORS), dtype=tf.float32)
    )
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

# 載入資料 (X_all需先製作好，shape: (N, 32, 11))
X_all = np.load(f'{MODEL_DIR}/X_all.npy')

train_ds = diffusion_dataset(X_all, BATCH_SIZE, T, alphas_cumprod, D)
steps_per_epoch = len(X_all) // BATCH_SIZE

# 訓練 Stage1 預訓練模型
history = encoder_full.fit(
    train_ds,
    epochs=EPOCHS_STAGE1,
    steps_per_epoch=steps_per_epoch,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)]
)

# 建立embedding模型，取倒數第二Conv1D層輸出後GlobalAveragePooling1D
embedding_layer_name = encoder_full.layers[-2].name
embedding_output = encoder_full.get_layer(embedding_layer_name).output
embedding_gap = layers.GlobalAveragePooling1D()(embedding_output)
encoder_embedding = Model(encoder_full.inputs, embedding_gap)

# 儲存模型，建議用.keras格式避免legacy warning
encoder_full.save(f'{MODEL_DIR}/stage1_encoder_full.keras')
encoder_embedding.save(f'{MODEL_DIR}/stage1_encoder_embedding.keras')

print("✅ Stage1 預訓練完成，模型與embedding模型已儲存。")



Epoch 1/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.8330
Epoch 2/300


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3345
Epoch 3/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3018
Epoch 4/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2804
Epoch 5/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2722
Epoch 6/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2562
Epoch 7/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2386
Epoch 8/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2275
Epoch 9/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2273
Epoch 10/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2154
Epoch 11/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2163
Epoch 12/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2185
Epoch 13/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2191
Epoch 14/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2126
Epoch 15/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2112
Epoch 16/300
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

Stage 2 聯合模型訓練與儲存

In [ ]:
# ===== 必要套件 =====
!pip install optuna tensorflow -q

import os
import joblib
import optuna
import numpy as np
import tensorflow as tf
import random
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import AdamW
from optuna.integration import TFKerasPruningCallback

# ===== 固定 SEED，確保重現 =====
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)

# ===== 參數設定 =====
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'
SEQ_LEN, D, T = 32, 16, 1000

# ===== 載入資料 =====
X_all = np.load(f'{MODEL_DIR}/X_all.npy')         # shape: (N, 32, 11)
y_all = np.load(f'{MODEL_DIR}/y_all.npy')         # shape: (N,)
unit_all = np.load(f'{MODEL_DIR}/unit_all.npy')   # shape: (N,)

# ===== 分組：80 engines 訓練 / 20 engines 驗證 =====
train_units = set(range(1, 81))
val_units = set(range(81, 101))

X_train, y_train, X_val, y_val = [], [], [], []
for x, y, u in zip(X_all, y_all, unit_all):
    if u in train_units:
        X_train.append(x)
        y_train.append(y)
    else:
        X_val.append(x)
        y_val.append(y)
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)

# ===== Diffusion timestep embedding =====
def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)

t_train = np.random.randint(1, T + 1, size=len(X_train))
tau_train = get_timestep_embedding_np(t_train, D)
t_val = np.random.randint(1, T + 1, size=len(X_val))
tau_val = get_timestep_embedding_np(t_val, D)

# ===== 載入 Stage1 Encoder embedding 模型 =====
encoder = tf.keras.models.load_model(f'{MODEL_DIR}/stage1_encoder_embedding.keras', compile=False)
encoder.trainable = True  # 可微調

# ===== RUL 預測 MLP 架構 =====
def build_rul_predictor(input_dim, hidden_units, dropout, activation):
    inputs = Input(shape=(input_dim,))
    x = inputs
    for h in hidden_units:
        x = layers.Dense(h, activation=activation)(x)
        x = layers.Dropout(dropout)(x)
    output = layers.Dense(1)(x)
    return Model(inputs, output)

# ===== Optuna objective =====
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 5e-4, log=True)
    dropout = trial.suggest_float("dropout", 0.1, 0.2)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    hidden_units = [trial.suggest_int(f"n_units_l{i}", 32, 128, step=32) for i in range(num_layers)]
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    activation = trial.suggest_categorical("activation", ["relu", "gelu", "swish"])
    lambda_high = trial.suggest_float("lambda_high", 0.01, 0.1)
    patience_es = trial.suggest_int("patience_es", 10, 20)
    patience_rlr = trial.suggest_int("patience_rlr", 3, 7)
    def custom_rul_loss(lambda_high=lambda_high):
        def loss(y_true, y_pred):
            base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
            high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
            return base + lambda_high * high_bias
        return loss

    input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]))
    input_t = Input(shape=(D,))
    encoded = encoder([input_x, input_t])
    mlp = build_rul_predictor(encoded.shape[-1], hidden_units, dropout, activation)
    output = mlp(encoded)

    joint_model = Model([input_x, input_t], output)
    joint_model.compile(optimizer=AdamW(learning_rate=learning_rate), loss=custom_rul_loss())

    joint_model.fit([X_train, tau_train], y_train,
                    validation_data=([X_val, tau_val], y_val),
                    epochs=200, batch_size=batch_size, verbose=0,
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(patience=patience_es, restore_best_weights=True),
                        tf.keras.callbacks.ReduceLROnPlateau(patience=patience_rlr, factor=0.5, min_lr=1e-7),
                        TFKerasPruningCallback(trial, "val_loss")
                    ])
    y_pred = joint_model.predict([X_val, tau_val], batch_size=batch_size).flatten()
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

# ===== 超參數搜尋 =====
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=GLOBAL_SEED))
study.optimize(objective, n_trials=100, timeout=1800)

# ===== 重訓最佳模型 =====
best = study.best_params
best_hidden = [best[f"n_units_l{i}"] for i in range(2)]

input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]))
input_t = Input(shape=(D,))
encoded = encoder([input_x, input_t])
mlp = build_rul_predictor(encoded.shape[-1], best_hidden, best['dropout'], best['activation'])
output = mlp(encoded)

def custom_rul_loss(lambda_high=best['lambda_high']):
    def loss(y_true, y_pred):
        base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

final_model = Model([input_x, input_t], output)
final_model.compile(optimizer=AdamW(learning_rate=best['learning_rate']), loss=custom_rul_loss())

final_model.fit([X_train, tau_train], y_train,
                validation_data=([X_val, tau_val], y_val),
                epochs=80,
                batch_size=best['batch_size'], verbose=2,
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(patience=best['patience_es'], restore_best_weights=True),
                    tf.keras.callbacks.ReduceLROnPlateau(patience=best['patience_rlr'], factor=0.5, min_lr=1e-7)
                ])

final_model.save(f'{MODEL_DIR}/stage2_joint_model_best.h5')

# ===== 驗證集結果評估 =====
y_pred = final_model.predict([X_val, tau_val], batch_size=best['batch_size']).flatten()
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"✅ 最佳參數: {best}")
print(f"✅ 驗證集最小 RMSE (piecewise RUL): {rmse:.4f}")
print(f"📁 模型已儲存至: {MODEL_DIR}/stage2_joint_model_best.h5")

# ===== 儲存 Optuna 搜尋結果 =====
df_trials = study.trials_dataframe()
df_trials.to_csv(f"{MODEL_DIR}/optuna_search_results.csv", index=False)
print(f"✅ Optuna 搜尋結果已儲存至: {MODEL_DIR}/optuna_search_results.csv")


[I 2025-06-01 06:27:14,969] A new study created in memory with name: no-name-2c8b7132-e8aa-40b7-90c1-d08aec2363bf


X_train: (13658, 32, 11) y_train: (13658,)
X_val: (3873, 32, 11) y_val: (3873,)
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


[I 2025-06-01 06:28:48,052] Trial 0 finished with value: 20.094353697919956 and parameters: {'learning_rate': 0.00018272261776066238, 'dropout': 0.19507143064099164, 'num_layers': 3, 'n_units_l0': 96, 'n_units_l1': 32, 'n_units_l2': 32, 'batch_size': 32, 'activation': 'gelu', 'lambda_high': 0.029110519961044856, 'patience_es': 12, 'patience_rlr': 3}. Best is trial 0 with value: 20.094353697919956.


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


[I 2025-06-01 06:29:33,865] Trial 1 finished with value: 16.847217436465954 and parameters: {'learning_rate': 0.00016317596956423415, 'dropout': 0.1524756431632238, 'num_layers': 2, 'n_units_l0': 64, 'n_units_l1': 96, 'batch_size': 128, 'activation': 'relu', 'lambda_high': 0.06331731119758383, 'patience_es': 10, 'patience_rlr': 6}. Best is trial 1 with value: 16.847217436465954.


...

[I 2025-06-01 06:45:56,827] Trial 23 finished with value: 15.868528473770073 and parameters: {'learning_rate': 0.00033554281199923075, 'dropout': 0.16654475480996112, 'num_layers': 4, 'n_units_l0': 128, 'n_units_l1': 128, 'n_units_l2': 96, 'n_units_l3': 128, 'batch_size': 16, 'activation': 'swish', 'lambda_high': 0.03581030592961036, 'patience_es': 16, 'patience_rlr': 6}. Best is trial 3 with value: 14.75894669576252.


243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


[I 2025-06-01 06:47:25,692] Trial 24 finished with value: 17.28928314081985 and parameters: {'learning_rate': 0.00026864837944213653, 'dropout': 0.18124410035834423, 'num_layers': 2, 'n_units_l0': 128, 'n_units_l1': 96, 'batch_size': 16, 'activation': 'swish', 'lambda_high': 0.01502652938440592, 'patience_es': 14, 'patience_rlr': 6}. Best is trial 3 with value: 14.75894669576252.
[I 2025-06-01 06:48:40,462] Trial 25 pruned. Trial was pruned at epoch 27.
[I 2025-06-01 06:49:54,948] Trial 26 pruned. Trial was pruned at epoch 26.
[I 2025-06-01 06:50:11,340] Trial 27 pruned. Trial was pruned at epoch 2.
[I 2025-06-01 06:50:33,941] Trial 28 pruned. Trial was pruned at epoch 20.
[I 2025-06-01 06:51:14,537] Trial 29 pruned. Trial was pruned at epoch 25.
[I 2025-06-01 06:51:49,442] Trial 30 pruned. Trial was pruned at epoch 10.
[I 2025-06-01 06:52:30,513] Trial 31 pruned. Trial was pruned at epoch 12.
[I 2025-06-01 06:52:46,984] Trial 32 pruned. Trial was pruned at epoch 2.
[I 2025-06-01 06:53

Epoch 1/80
854/854 - 8s - 10ms/step - loss: 734.2474 - val_loss: 493.2842 - learning_rate: 2.4106e-04
Epoch 2/80
854/854 - 2s - 3ms/step - loss: 242.6398 - val_loss: 391.5890 - learning_rate: 2.4106e-04
Epoch 3/80
854/854 - 2s - 3ms/step - loss: 201.5226 - val_loss: 367.9125 - learning_rate: 2.4106e-04
Epoch 4/80
854/854 - 2s - 3ms/step - loss: 182.8866 - val_loss: 373.9537 - learning_rate: 2.4106e-04
Epoch 5/80
854/854 - 2s - 3ms/step - loss: 174.0871 - val_loss: 361.5573 - learning_rate: 2.4106e-04
Epoch 6/80
854/854 - 2s - 3ms/step - loss: 167.2253 - val_loss: 396.4471 - learning_rate: 2.4106e-04
Epoch 7/80
854/854 - 2s - 3ms/step - loss: 161.0314 - val_loss: 322.3735 - learning_rate: 2.4106e-04
Epoch 8/80
854/854 - 2s - 3ms/step - loss: 156.8647 - val_loss: 384.6866 - learning_rate: 2.4106e-04
Epoch 9/80
854/854 - 2s - 3ms/step - loss: 150.5642 - val_loss: 345.8916 - learning_rate: 2.4106e-04
Epoch 10/80
854/854 - 2s - 3ms/step - loss: 145.7627 - val_loss: 383.7415 - learning_rate:

243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
✅ 最佳參數: {'learning_rate': 0.00024106495902171608, 'dropout': 0.1184854455525527, 'num_layers': 4, 'n_units_l0': 128, 'n_units_l1': 128, 'n_units_l2': 128, 'n_units_l3': 96, 'batch_size': 16, 'activation': 'gelu', 'lambda_high': 0.08458637582367365, 'patience_es': 13, 'patience_rlr': 4}
✅ 驗證集最小 RMSE (piecewise RUL): 17.1066
📁 模型已儲存至: /content/drive/MyDrive/rul_diffusion_fd001/stage2_joint_model_best.h5
✅ Optuna 搜尋結果已儲存至: /content/drive/MyDrive/rul_diffusion_fd001/optuna_search_results.csv


Test資料集預處理

In [ ]:
import numpy as np
import pandas as pd
import joblib
import os

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'

# === 1. 固定 SEQ_LEN ===
SEQ_LEN = 32
print("使用固定 SEQ_LEN:", SEQ_LEN)

# === 2. 載入 scaler、特徵名（這裡是11個特徵）===
scaler = joblib.load(f'{MODEL_DIR}/scaler_preprocessed.joblib')
feature_names = joblib.load(f'{MODEL_DIR}/feature_names.pkl')  # 11個特徵名列表

# === 3. 載入 Test 原始資料 ===
columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]
test = pd.read_csv('/content/cmapss_data/CMaps/test_FD001.txt', sep='\s+', header=None)
test.columns = columns
rul_truth = pd.read_csv('/content/cmapss_data/CMaps/RUL_FD001.txt', header=None, names=['RUL'])

# === 4. 單一 alpha=0.2 指數平滑 ===
def exponential_smoothing(series, alpha=0.2):
    result = [series.iloc[0]]
    for n in range(1, len(series)):
        result.append(alpha * series.iloc[n] + (1 - alpha) * result[-1])
    return pd.Series(result, index=series.index)

for col in feature_names:
    test[col] = test.groupby('unit')[col].transform(lambda x: exponential_smoothing(x, alpha=0.2))

# === 5. 特徵正規化 ===
test[feature_names] = scaler.transform(test[feature_names])

# === 6. 產生 Test 視窗 ===
X_test_list, timestep_list, units_list = [], [], []
for unit in sorted(test['unit'].unique()):
    df_unit = test[test['unit'] == unit]
    arr = df_unit[feature_names].values
    if len(arr) >= SEQ_LEN:
        X_test_list.append(arr[-SEQ_LEN:])
        timestep_list.append(df_unit['time'].values[-1])
        units_list.append(unit)
X_test = np.stack(X_test_list).astype(np.float32)

# === 7. Timestep embedding ===
def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)

D = 16
tau_test = get_timestep_embedding_np(timestep_list, D)

# === 8. 載入真實RUL（依unit排序） ===
y_test = rul_truth['RUL'].values  # 確保跟units_list順序一致，若不一致需對齊

# === 9. 儲存資料 ===
np.save(f'{MODEL_DIR}/X_test.npy', X_test)
np.save(f'{MODEL_DIR}/tau_test.npy', tau_test)
np.save(f'{MODEL_DIR}/units_list.npy', np.array(units_list))
np.save(f'{MODEL_DIR}/y_test.npy', y_test)

print(f"✅ Test 視窗數: {len(X_test)}")
print(f"🧪 對齊 unit 數量: {len(np.unique(units_list))}")

使用固定 SEQ_LEN: 32
✅ Test 視窗數: 99
🧪 對齊 unit 數量: 99


Test驗證、計算RMSE與NASA Score、繪圖並存檔

In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'

# === 1. 載入 TestSet 前處理資料 ===
X_test = np.load(f'{MODEL_DIR}/X_test.npy')
tau_test = np.load(f'{MODEL_DIR}/tau_test.npy')
units_list = np.load(f'{MODEL_DIR}/units_list.npy').astype(int)  # 必須是 int 型態

# === 1-2. 載入 NASA ground-truth RUL 並與 units_list 對齊 ===
rul_truth = pd.read_csv('/content/cmapss_data/CMaps/RUL_FD001.txt', header=None, names=['RUL'])
y_true = rul_truth.loc[units_list - 1, 'RUL'].values

# === 2. 自動抓訓練時最佳 lambda_high ===
optuna_result_csv = f'{MODEL_DIR}/optuna_search_results.csv'
optuna_df = pd.read_csv(optuna_result_csv)
if 'params_lambda_high' in optuna_df.columns:
    best_trial = optuna_df.loc[optuna_df['value'].idxmin()]
    best_lambda_high = best_trial['params_lambda_high']
else:
    best_lambda_high = 0.1  # 預設值

def custom_rul_loss(lambda_high=best_lambda_high):
    def loss(y_true, y_pred):
        base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

# === 3. 載入訓練好的 Stage2 聯合模型 ===
joint_model = load_model(
    f'{MODEL_DIR}/stage2_joint_model_best.h5',
    custom_objects={'loss': custom_rul_loss()}
)

# === 4. 預測 ===
y_pred = joint_model.predict([X_test, tau_test], verbose=1).flatten()
y_pred = np.clip(y_pred, 0, 130)

# === 5. NASA Score 函數 ===
def nasa_score(y_true, y_pred):
    score = 0
    for true, pred in zip(y_true, y_pred):
        d = pred - true
        if d < 0:
            score += np.exp(-d / 13) - 1
        else:
            score += np.exp(d / 10) - 1
    return score

# === 6. 計算分數 ===
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
score = nasa_score(y_true, y_pred)
print(f'✅ Test RMSE: {rmse:.4f}')
print(f'✅ Test NASA Score: {score:.4f}')

# === 7. 儲存預測結果 ===
df_out = pd.DataFrame({
    'unit': units_list,
    'True_RUL': y_true,
    'Pred_RUL': y_pred,
    'Error': y_pred - y_true
})
df_out.to_csv(f'{MODEL_DIR}/test_pred_result.csv', index=False)
print(f"📁 預測結果已儲存：{MODEL_DIR}/test_pred_result.csv")

# === 8. 圖表: 預測 vs 真實 RUL ===
plt.figure(figsize=(8, 5))
plt.plot(y_true, label='True RUL', marker='o', alpha=0.8)
plt.plot(y_pred, label='Predicted RUL', marker='x', alpha=0.8)
plt.xlabel('Test Unit (Engine)')
plt.ylabel('RUL')
plt.legend()
plt.title(f'Predicted vs. True RUL (Test)\nRMSE={rmse:.2f} | NASA Score={score:.2f}')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_pred_vs_true_rul.png')
plt.close()

# === 9. 殘差分布圖 (Error vs True RUL) ===
plt.figure(figsize=(7,5))
plt.scatter(y_true, y_pred - y_true, alpha=0.75, c='royalblue', edgecolor='k')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('True RUL')
plt.ylabel('Prediction Error (Predicted - True)')
plt.title('Residual Distribution (Error vs True RUL)')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_residual_scatter.png')
plt.close()

# === 10. 預測誤差直方圖 ===
plt.figure(figsize=(7,4))
plt.hist(y_pred - y_true, bins=25, color='skyblue', edgecolor='black')
plt.title('Prediction Error Histogram')
plt.xlabel('Prediction Error (Predicted - True)')
plt.ylabel('Count')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_error_histogram.png')
plt.close()

# === 11. t-SNE 可視化 Stage1 Encoder Embedding 特徵（可選） ===
try:
    encoder = load_model(f'{MODEL_DIR}/stage1_encoder_embedding.keras', compile=False)
    features = encoder.predict([X_test, tau_test])
    features_2d = TSNE(n_components=2, random_state=42).fit_transform(features)
    plt.figure(figsize=(7,6))
    sc = plt.scatter(features_2d[:,0], features_2d[:,1], c=y_true, cmap='viridis', s=20)
    plt.colorbar(sc, label='True RUL')
    plt.title("t-SNE of Encoder Embedding Features (Test set)")
    plt.xlabel("t-SNE Dim 1")
    plt.ylabel("t-SNE Dim 2")
    plt.savefig(f'{MODEL_DIR}/test_encoder_tsne.png')
    plt.close()
except Exception as e:
    print(f"[t-SNE 可視化失敗] {e}")

print(f"📈 圖片已儲存：\n{MODEL_DIR}/test_pred_vs_true_rul.png\n{MODEL_DIR}/test_residual_scatter.png\n{MODEL_DIR}/test_error_histogram.png\n{MODEL_DIR}/test_encoder_tsne.png")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/step
✅ Test RMSE: 16.4323
✅ Test NASA Score: 449.7096
📁 預測結果已儲存：/content/drive/MyDrive/rul_diffusion_fd001/test_pred_result.csv


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
📈 圖片已儲存：
/content/drive/MyDrive/rul_diffusion_fd001/test_pred_vs_true_rul.png
/content/drive/MyDrive/rul_diffusion_fd001/test_residual_scatter.png
/content/drive/MyDrive/rul_diffusion_fd001/test_error_histogram.png
/content/drive/MyDrive/rul_diffusion_fd001/test_encoder_tsne.png
